# Segmenting and Clustering Neighborhoods in Toronto

Author: Philippe Zahlen

In [62]:
# Loading dependencies
import pandas as pd
import numpy as np

In [63]:
pip install folium

     |████████████████████████████████| 94 kB 7.2 MB/s  eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [64]:
import folium

In [65]:
# Copy-paste to Excel from Wikipedia; Save-as csv file; Fileupload to githup
# Loading data to a pandas dataframe
file_name = 'https://raw.githubusercontent.com/pzahlen/Coursera_Capstone/master/List_of_postal_codes_of_Toronto.csv'
df = pd.read_csv(file_name, sep = ';')
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M1B,Scarborough,"Malvern, Rouge"
2,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
3,M1E,Scarborough,"Guildwood, Morningside, West Hill"
4,M1G,Scarborough,Woburn


In [66]:
# Dropping the rows where Borough is not assigned
df1 = df[df.Borough != 'Not assigned']
df1.head()
# No more cleaning necessary as there is no cell which has a burough but a not assigned in the neighbourhood

,Postal Code,Borough,Neighbourhood
1,M1B,Scarborough,"Malvern, Rouge"
2,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
3,M1E,Scarborough,"Guildwood, Morningside, West Hill"
4,M1G,Scarborough,Woburn
5,M1H,Scarborough,Cedarbrae


In [67]:
df1.shape

(103, 3)

In [68]:
# Importing csv of the geographical coordinates of each postal code in Canada
df_geo_coordinates = pd.read_csv('https://cocl.us/Geospatial_data')
df_geo_coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [69]:
# Merging both of the tables based on the postal code
df2 = pd.merge(df1, df_geo_coordinates, on = 'Postal Code')
df2.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [70]:
# Getting just the rows which contains Toronto in their borough
df3 = df2[df2['Borough'].str.contains('Toronto', regex=False)]
df3.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [71]:
# Visualizing map
map_of_toronto = folium.Map(location=[43.651070,-79.347015], zoom_start = 10)

for lat,lng,borough,neighbourhood in zip(df3['Latitude'],df3['Longitude'],df3['Borough'],df3['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_of_toronto)
map_of_toronto

In [77]:
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

In [73]:
# KNN clustering with k = 6
k = 6
toronto_clustering = df3.drop(['Postal Code','Borough','Neighbourhood'], 1)
kmeans = KMeans(n_clusters = k,random_state = 0).fit(toronto_clustering)
kmeans.labels_
df3.insert(0, 'Cluster Labels', kmeans.labels_)

In [75]:
# Displaying
df3.head()

,Cluster Labels,Postal Code,Borough,Neighbourhood,Latitude,Longitude
37,5,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,5,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
43,5,M4M,East Toronto,Studio District,43.659526,-79.340923
44,2,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [78]:
map_clusters = folium.Map(location=[43.651070,-79.347015], zoom_start = 10)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, neighbourhood, cluster in zip(df3['Latitude'], df3['Longitude'], df3['Neighbourhood'], df3['Cluster Labels']):
    label = folium.Popup(' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters